### Exploring the built environment/SDCI datasets

In [ ]:
# Find all *.csv files in the data/ directory. Load each into a pandas DataFrame,
# and create a dictionary mapping filenames (without extension) to DataFrames.

import pandas as pd
import os
data_dir = '../data'
dataframes = {}
for filename in os.listdir(data_dir):
    if filename.endswith('.jsonl'):
        filepath = os.path.join(data_dir, filename)
        df_name = os.path.splitext(filename)[0]
        dataframes[df_name] = pd.read_json(filepath, lines=True)

# Now `dataframes` is a dictionary where keys are filenames without extensions
# and values are the corresponding pandas DataFrames.
for k in dataframes.keys():
    print(k)

In [ ]:
# Let's take a look at the building_permits DataFrame
building_permits_df = dataframes['building_permits']

# How many rows and columns does it have?
print(building_permits_df.shape)  # (number of rows, number of columns)

# How many have applieddate in every year since 2000?
for year in range(2000, 2026):
    applied_year = building_permits_df[building_permits_df['applieddate'].fillna('').str.startswith(str(year))]
    print(f"{year}: {len(applied_year)}")

In [ ]:
# Print the first few rows to inspect, and don't truncate columns or wrap.
# Display all columns.
# Print the first few rows to inspect, and don't truncate columns or wrap.
# Display all columns.
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(building_permits_df.head())

# How many completed permits are there? (statuscurrent is "Completed")
completed_permits = building_permits_df[building_permits_df['statuscurrent'] == 'Completed']
print(f"Number of completed permits: {len(completed_permits)}")

# Take a look at top types

top_permit_types = building_permits_df['permittypemapped'].value_counts()
print("\n--------------------------------")
print("Top most common permit types among completed permits:")
print(top_permit_types)

top_permit_desc = building_permits_df['permittypedesc'].value_counts()
print("\n--------------------------------")
print("Top most common permit desc among completed permits:")
print(top_permit_desc)

top_permit_classes = building_permits_df['permitclass'].value_counts()
print("\n--------------------------------")
print("Top 10 most common permit classes:")
print(top_permit_classes)

top_permit_class_mapped = building_permits_df['permitclassmapped'].value_counts()
print("\n--------------------------------")
print("Top 10 most common permit classes:")
print(top_permit_class_mapped)


In [ ]:
import folium
import json
from folium.plugins import MarkerCluster
# from IPython.display import display, HTML

# Map all permits that were applied for in 2025
applied_2025 = building_permits_df[building_permits_df['applieddate'].fillna('').str.startswith('2025')]
print(f"\nNumber of permits applied for in 2025: {len(applied_2025)}")

# Create a folium map centered around Seattle
seattle_map = folium.Map(location=[47.6062, -122.3321], zoom_start=10, tiles='CartoDB positron')
marker_cluster = MarkerCluster().add_to(seattle_map)

# Add markers for each permit
for idx, row in applied_2025.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    permit_class = row['permitclass']
    link_url = row.get("link", {}).get("url", "#")
    if pd.notnull(lat) and pd.notnull(lon) and permit_class == 'Commercial':
        folium.Marker(
            location=[lat, lon],
            popup=f"Permit Type: {row['permittypemapped']}<br>Address: {row['originaladdress1']}<br><a href='{link_url}' target='_blank'>View Permit</a>",
        ).add_to(marker_cluster)

# Display the map
seattle_map